# Shark Attacks - Exploratory Data Analysis

**Project:** Shark Attacks Data Analysis  
**Author:** Data Science Bootcamp - Ironhack  
**Date:** January 2026

## Objective
This notebook performs exploratory data analysis (EDA) on the Global Shark Attack File (GSAF) dataset to understand:
- Dataset structure and quality
- Distribution of key variables
- Data patterns and relationships
- Initial insights for hypothesis formulation

## 1. Setup and Data Loading

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Import custom modules
import sys
sys.path.append('..')
from src import set_plot_style

# Set visualization style
set_plot_style()

In [ ]:
# Load cleaned data (already processed)
df = pd.read_csv('../data/shark_attacks_cleaned.csv')

## 2. Dataset Overview

In [ ]:
# Dataset dimensions
df.shape

In [ ]:
# Column names and types
df.dtypes

In [ ]:
# First few rows
df.head(10)

In [ ]:
# Summary statistics for numeric columns
df.describe()

**Initial Observations:**
- Dataset contains ~7,000 shark attack incidents
- Mix of categorical and numerical variables
- Data spans several centuries of recorded attacks

### Data Quality Assessment

In [ ]:
# Missing value analysis
missing_data = pd.DataFrame({
    'Missing_Count': df.isnull().sum(),
    'Missing_Percentage': (df.isnull().sum() / len(df) * 100).round(2)
}).sort_values('Missing_Count', ascending=False)

missing_data[missing_data['Missing_Count'] > 0]

In [ ]:
# Visualize missing data patterns
fig, ax = plt.subplots(figsize=(10, 6))
missing_cols = missing_data[missing_data['Missing_Count'] > 0].head(10)
missing_cols['Missing_Percentage'].plot(kind='barh', color='coral', ax=ax)
ax.set_title('Top 10 Columns with Missing Data', fontsize=14, fontweight='bold')
ax.set_xlabel('Missing Data (%)', fontsize=11)
ax.set_ylabel('Column', fontsize=11)
ax.grid(axis='x', alpha=0.3)
plt.tight_layout()
plt.show()

**Data Quality Insights:**
- Some historical data has expected gaps (Age, Species identification)
- Core variables (Country, Year, Activity) have good completeness
- Missing data reflects real-world challenges in attack documentation

## 3. Geographic Distribution

Where do shark attacks occur most frequently?

In [ ]:
# Top countries by attack count
top_countries = df['Country'].value_counts().head(15)
top_countries

In [ ]:
# Calculate concentration in top 3 countries
top3_count = top_countries.head(3).sum()
top3_percentage = (top3_count / len(df)) * 100

pd.DataFrame({
    'Metric': ['Top 3 Countries Total', 'Percentage of All Attacks'],
    'Value': [f"{top3_count:,}", f"{top3_percentage:.1f}%"]
})

In [ ]:
# Visualize geographic distribution
fig, ax = plt.subplots(figsize=(12, 7))
top_countries.plot(kind='bar', color='steelblue', ax=ax)
ax.set_title('Top 15 Countries by Shark Attacks', fontsize=16, fontweight='bold')
ax.set_xlabel('Country', fontsize=12)
ax.set_ylabel('Number of Attacks', fontsize=12)
ax.grid(axis='y', alpha=0.3)
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

**Geographic Insights:**
- Extreme concentration in USA, Australia, and South Africa
- These 3 countries account for ~66% of all recorded attacks
- Reflects combination of:
  - Coastal geography and shark populations
  - High levels of water-based recreational activities
  - Strong reporting infrastructure

## 4. Temporal Patterns

How have shark attacks changed over time?

In [ ]:
# Year range and basic statistics
df['Year'].describe()

In [ ]:
# Filter for modern era (1900-2025)
df_temporal = df[df['Year'].notna() & (df['Year'] >= 1900) & (df['Year'] <= 2025)].copy()

# Attacks by decade
df_temporal['Decade'] = (df_temporal['Year'] // 10) * 10
attacks_by_decade = df_temporal['Decade'].value_counts().sort_index()

attacks_by_decade

In [ ]:
# Yearly trend
attacks_by_year = df_temporal.groupby('Year').size()

fig, ax = plt.subplots(figsize=(14, 6))
attacks_by_year.plot(kind='line', marker='o', markersize=3, color='darkred', linewidth=1.5, ax=ax)
ax.set_title('Shark Attacks Over Time (1900-2025)', fontsize=16, fontweight='bold')
ax.set_xlabel('Year', fontsize=12)
ax.set_ylabel('Number of Attacks', fontsize=12)
ax.grid(True, alpha=0.3)
ax.fill_between(attacks_by_year.index, attacks_by_year.values, alpha=0.2, color='darkred')
plt.tight_layout()
plt.show()

**Temporal Trends:**
- Clear upward trend in reported attacks since 1900
- Particularly sharp increase from 1950s onwards
- Likely driven by:
  - Population growth in coastal areas
  - Increased water sports participation
  - Improved reporting and documentation

## 5. Activity Analysis

What activities are people doing when attacks occur?

In [ ]:
# Top activities during attacks
top_activities = df['Activity'].value_counts().head(15)
top_activities

In [ ]:
# Calculate surfing + swimming percentage
surfing_swimming = df[df['Activity'].str.contains('Surfing|Swimming', case=False, na=False)]
surf_swim_pct = (len(surfing_swimming) / len(df)) * 100

pd.DataFrame({
    'Activity Group': ['Surfing', 'Swimming', 'Combined'],
    'Count': [
        len(df[df['Activity'].str.contains('Surfing', case=False, na=False)]),
        len(df[df['Activity'].str.contains('Swimming', case=False, na=False)]),
        len(surfing_swimming)
    ],
    'Percentage': [
        f"{len(df[df['Activity'].str.contains('Surfing', case=False, na=False)]) / len(df) * 100:.1f}%",
        f"{len(df[df['Activity'].str.contains('Swimming', case=False, na=False)]) / len(df) * 100:.1f}%",
        f"{surf_swim_pct:.1f}%"
    ]
})

In [ ]:
# Visualize activity distribution
fig, ax = plt.subplots(figsize=(12, 8))
top_activities.plot(kind='barh', color='coral', ax=ax)
ax.set_title('Top 15 Activities During Shark Attacks', fontsize=16, fontweight='bold')
ax.set_xlabel('Number of Attacks', fontsize=12)
ax.set_ylabel('Activity', fontsize=12)
ax.grid(axis='x', alpha=0.3)
plt.tight_layout()
plt.show()

**Activity Insights:**
- Surfing and swimming dominate attack scenarios
- Combined, they account for ~37% of all attacks
- Recreational activities more common than commercial (fishing, diving)
- Surface activities have higher visibility to sharks

## 6. Victim Demographics

Who are the victims of shark attacks?

### Gender Distribution

In [ ]:
# Gender breakdown
gender_counts = df['Sex'].value_counts()
gender_counts

In [ ]:
# Calculate gender ratio
male_count = gender_counts.get('M', 0)
female_count = gender_counts.get('F', 0)
ratio = male_count / female_count

pd.DataFrame({
    'Gender': ['Male', 'Female', 'Ratio (M:F)'],
    'Value': [
        f"{male_count:,} ({male_count/(male_count+female_count)*100:.1f}%)",
        f"{female_count:,} ({female_count/(male_count+female_count)*100:.1f}%)",
        f"{ratio:.1f}:1"
    ]
})

In [ ]:
# Visualize gender distribution
fig, ax = plt.subplots(figsize=(8, 8))
colors = ['#3498db', '#e74c3c']
gender_counts.plot(kind='pie', autopct='%1.1f%%', startangle=90, colors=colors, ax=ax)
ax.set_title('Gender Distribution of Shark Attack Victims', fontsize=16, fontweight='bold')
ax.set_ylabel('')
plt.tight_layout()
plt.show()

**Gender Insights:**
- Strong male predominance (~88% of victims)
- Male to female ratio approximately 7:1
- Likely reflects:
  - Higher male participation in water sports
  - Potentially different risk-taking behaviors
  - Historical gender patterns in coastal activities

### Age Distribution

In [ ]:
# Age statistics
df['Age'].describe()

In [ ]:
# Age group breakdown
df_age = df[df['Age'].notna()].copy()
df_age['Age_Group'] = pd.cut(df_age['Age'], 
                              bins=[0, 12, 18, 30, 50, 100],
                              labels=['Child (0-12)', 'Teen (13-18)', 'Young Adult (19-30)', 
                                      'Adult (31-50)', 'Senior (50+)'])

df_age['Age_Group'].value_counts().sort_index()

In [ ]:
# Visualize age distribution
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

# Histogram
ax1.hist(df_age['Age'], bins=30, color='skyblue', edgecolor='black', alpha=0.7)
ax1.axvline(df_age['Age'].mean(), color='red', linestyle='--', linewidth=2, 
            label=f'Mean: {df_age["Age"].mean():.1f}')
ax1.axvline(df_age['Age'].median(), color='green', linestyle='--', linewidth=2, 
            label=f'Median: {df_age["Age"].median():.1f}')
ax1.set_title('Age Distribution', fontsize=14, fontweight='bold')
ax1.set_xlabel('Age', fontsize=11)
ax1.set_ylabel('Frequency', fontsize=11)
ax1.legend()
ax1.grid(axis='y', alpha=0.3)

# Box plot
ax2.boxplot(df_age['Age'], vert=True, patch_artist=True,
            boxprops=dict(facecolor='lightcoral', alpha=0.7),
            medianprops=dict(color='darkred', linewidth=2))
ax2.set_title('Age Box Plot', fontsize=14, fontweight='bold')
ax2.set_ylabel('Age', fontsize=11)
ax2.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

**Age Demographics:**
- Median age: 24 years (peak water activity age)
- Mean age: 28 years
- Distribution is right-skewed (younger victims more common)
- Young adults (19-30) represent the highest risk group
- Aligns with active water sports participation demographics

## 7. Attack Outcomes

What are the consequences of shark attacks?

In [ ]:
# Fatality statistics
fatal_counts = df['Fatal Y/N'].value_counts()
fatality_rate = (fatal_counts.get('Y', 0) / fatal_counts.sum()) * 100

pd.DataFrame({
    'Outcome': ['Fatal', 'Non-Fatal', 'Fatality Rate'],
    'Value': [
        f"{fatal_counts.get('Y', 0):,}",
        f"{fatal_counts.get('N', 0):,}",
        f"{fatality_rate:.1f}%"
    ]
})

In [ ]:
# Visualize outcomes
fig, ax = plt.subplots(figsize=(8, 8))
colors = ['#2ecc71', '#e74c3c']
fatal_counts.plot(kind='pie', autopct='%1.1f%%', startangle=90, colors=colors, ax=ax,
                  labels=['Non-Fatal', 'Fatal'])
ax.set_title('Shark Attack Outcomes', fontsize=16, fontweight='bold')
ax.set_ylabel('')
plt.tight_layout()
plt.show()

**Outcome Analysis:**
- Majority of attacks (~77%) are non-fatal
- Fatality rate approximately 23%
- Survival factors include:
  - Proximity to medical care
  - Speed of emergency response
  - Nature of the attack
- Most attacks involve investigative bites rather than predation

## 8. Shark Species

Which shark species are involved in attacks?

In [ ]:
# Top species involved
species_col = 'Species ' if 'Species ' in df.columns else 'Species'
top_species = df[species_col].value_counts().head(15)
top_species

In [ ]:
# Visualize species distribution
fig, ax = plt.subplots(figsize=(12, 8))
top_species.plot(kind='barh', color='darkslategray', ax=ax)
ax.set_title('Top 15 Shark Species in Attacks', fontsize=16, fontweight='bold')
ax.set_xlabel('Number of Attacks', fontsize=12)
ax.set_ylabel('Species', fontsize=12)
ax.grid(axis='x', alpha=0.3)
plt.tight_layout()
plt.show()

**Species Observations:**
- White Shark (Great White) most commonly identified
- Tiger Shark and Bull Shark also frequently involved
- These three species share characteristics:
  - Large size and power
  - Coastal habitat overlap with humans
  - Curious, investigative behavior
- Many attacks have unconfirmed species (identification difficult)

## 9. EDA Summary & Key Findings

### Geographic Patterns
- **Extreme concentration:** USA, Australia, South Africa account for ~66% of attacks
- Clear hotspot regions suggest environmental and activity factors

### Temporal Trends
- **Significant increase** in attacks from 1950s onward
- Likely reflects reporting improvements and increased coastal activity
- Modern era shows consistent upward trend

### Activity Patterns
- **Surfing and swimming dominate** (~37% combined)
- Recreational activities more common than commercial
- Surface water activities highest risk

### Demographics
- **Strong male bias:** 87.5% of victims (7:1 ratio)
- **Young adult concentration:** Median age 24 years
- Peak risk group: Males aged 15-30

### Outcomes
- **Majority survivable:** ~77% non-fatal
- Fatality rate ~23%
- Medical response critical to survival

### Species
- **Top three:** White Shark, Tiger Shark, Bull Shark
- Large coastal species with investigative behavior
- Many incidents have uncertain identification

### Implications for Analysis
These patterns suggest clear, testable hypotheses:
1. Geographic hotspots can be quantified and predicted
2. Activity-based risk assessment is viable
3. Demographic patterns enable targeted interventions
4. Temporal trends inform resource allocation

**Next Step:** Proceed to hypothesis testing notebook to validate these observations with statistical rigor.